In [1]:
from psycopg2 import DatabaseError
from psycopg2.extras import DictCursor

from idb.postgres_backend.db import PostgresDB, RecordSet

db = PostgresDB()

In [2]:
from __future__ import print_function
import feedparser
from pyquery import PyQuery as pq
import argparse

In [3]:
feed_to_parse = 'https://ipt.vertnet.org/rss.do'

In [4]:
if not feed_to_parse.startswith('http'):
    print ()
    print ("* non-HTTP feed supplied, assuming local file. *")

feed = feedparser.parse(feed_to_parse)


def get_title(entry):
    if "title" in entry:
        return(entry["title"])
    else:
        return("NO TITLE FOUND")

def get_pubDate(entry):
    if "published" in entry:
        return (entry["published"])
    else:
        return ("NO PUBLISHED DATE FOUND")


def get_id(entry):
    if "id" in entry:
        return (entry["id"])
    else:
        return ("NO ID FOUND")

def get_dataset_link(entry):
    if "ipt_dwca" in entry:
        return (entry["ipt_dwca"])
    elif "link" in entry:
        return (entry["link"])
    else:
        return ("NO DATASET LINK FOUND")

def get_eml_link(entry):
    if "ipt_eml" in entry:
        return (entry["ipt_eml"])
    elif "emllink" in entry:
        return (entry["emllink"])
    else:
        return ("NO EML LINK FOUND")


In [5]:
print (feed_to_parse)
if "title" in feed['feed']:
    print (feed['feed']['title'])
else:
    print ("Feed has no TITLE.")


https://ipt.vertnet.org/rss.do
IPT - Hosted by VertNet


In [6]:

for entry in feed.entries:
    entry_title = ""
    entry_pubDate = ""
    entry_id = ""
    entry_dataset_link = ""
    entry_eml_link = ""
    # feedparser converts many common fields into normalized names. Examples:
    #   guid --> id
    #   pubDate --> published
    #
    # Fields that contain colons such as ipt:dwca and ipt:eml get underscored to ipt_dwca and ipt_eml
    #
    # The actual IPT guid field is not visible as a normalized field since another id field is used.
    # However, the id is embedded in the middle of the id url so human can pluck it out if needed.


    print ("title:        ", get_title(entry).encode('utf-8').strip())
    print ("published:    ", get_pubDate(entry).encode('utf-8').strip())
    print ("id:           ", get_id(entry).encode('utf-8').strip())
    print ("dataset link: ", get_dataset_link(entry).encode('utf-8').strip())
    print ("eml link:     ", get_eml_link(entry).encode('utf-8').strip())


title:         Jurica-Suchy Nature Museum at Benedictine University Herbarium Specimens (Arctos) - Version 1.33
published:     Fri, 10 May 2024 11:58:18 +0000
id:            d39570b9-2972-4dbb-b41f-6826823176ae/v1.33
dataset link:  https://ipt.vertnet.org/archive.do?r=jsnm_herb
eml link:      https://ipt.vertnet.org/eml.do?r=jsnm_herb
title:         Jurica-Suchy Nature Museum at Benedictine University Egg Specimens (Arctos) - Version 1.33
published:     Fri, 10 May 2024 11:56:18 +0000
id:            f269f7b1-ad56-45d0-8641-8bc0e84b411a/v1.33
dataset link:  https://ipt.vertnet.org/archive.do?r=jsnm_egg
eml link:      https://ipt.vertnet.org/eml.do?r=jsnm_egg
title:         Jurica-Suchy Nature Museum at Benedictine University Fossil Specimens (Arctos) - Version 1.34
published:     Fri, 10 May 2024 11:54:31 +0000
id:            ebab4eda-c07d-4a40-a66a-d0f5b78a1235/v1.34
dataset link:  https://ipt.vertnet.org/archive.do?r=jsnm_paleo
eml link:      https://ipt.vertnet.org/eml.do?r=jsnm_pale

In [7]:
rs_matches = {}

for entry in feed.entries:
    entry_id = get_id(entry).encode('utf-8').strip() 
    if not entry_id.startswith('http') and len(entry_id.split('/v')) > 1:
        entry_id = entry_id.split('/v')[0]
        rs_sql = "select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%{0}%'".format(entry_id)
        rs = db.fetchall(rs_sql)
        rs_matches[get_id(entry).encode('utf-8').strip()] = rs
    else:
        entry_id = entry_id.split('/v')[0]
        entry_id = entry_id.replace('https://ipt.vertnet.org', 'http://ipt.vertnet.org:8080/ipt')
        print(entry_id)
        rs_sql = "select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%{0}%'".format(entry_id)
        print(rs_sql)
        rs = db.fetchall(rs_sql)
        rs_matches[entry_id] = rs
        

http://ipt.vertnet.org:8080/ipt/resource?id=uam_herp_ggbn
select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%http://ipt.vertnet.org:8080/ipt/resource?id=uam_herp_ggbn%'
http://ipt.vertnet.org:8080/ipt/resource?id=uam_inv_ggbn
select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%http://ipt.vertnet.org:8080/ipt/resource?id=uam_inv_ggbn%'
http://ipt.vertnet.org:8080/ipt/resource?id=msb_mammal_ggbn
select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%http://ipt.vertnet.org:8080/ipt/resource?id=msb_mammal_ggbn%'
http://ipt.vertnet.org:8080/ipt/resource?id=uam_mamm_ggbn
select * from recordsets where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%http://ipt.vertnet.org:8080/ipt/resource?id=uam_mamm_ggbn%'
http:/

In [8]:
rs_matches
    

{'0150c79f-35f3-447f-a40f-24ee9f1fb9d3/v6.2': [[150642L,
   None,
   '842a2bb5-d705-4d6c-8401-abf3ca28c05d',
   u'DMNH Eggs - Version 6.1',
   [u'0150c79f-35f3-447f-a40f-24ee9f1fb9d3'],
   u'https://ipt.vertnet.org/eml.do?r=dmnh_eggs',
   u'https://ipt.vertnet.org/archive.do?r=dmnh_eggs',
   False,
   datetime.datetime(2024, 1, 3, 20, 33, 23, 728602),
   datetime.datetime(2024, 3, 20, 16, 6, 33, 801248),
   datetime.datetime(2024, 1, 31, 21, 7, 12),
   None,
   None,
   None,
   None,
   False,
   None]],
 '01bdf3ea-140a-4d45-a27c-aeca97662aea/v1.15': [[150451L,
   None,
   '842a2bb5-d705-4d6c-8401-abf3ca28c05d',
   u'Abilene Christian University Insect Specimens (Arctos) - Version 1.9',
   [u'01bdf3ea-140a-4d45-a27c-aeca97662aea'],
   u'https://ipt.vertnet.org/eml.do?r=acunhc_ento',
   u'https://ipt.vertnet.org/archive.do?r=acunhc_ento',
   False,
   datetime.datetime(2024, 1, 3, 20, 33, 23, 728602),
   datetime.datetime(2024, 3, 20, 16, 6, 33, 709395),
   datetime.datetime(2024, 3, 4

In [9]:
current_rs = {}

for key in rs_matches.keys():
    if not key.startswith('http') and len(key.split('/v')) > 1:
        key = key.split('/v')[0]
    rs_sql = "select id, uuid, publisher_uuid, name, recordids, eml_link, file_link, ingest, first_seen, last_seen, pub_date, file_harvest_date, file_harvest_etag, eml_harvest_date, eml_harvest_etag, ingest_is_paused, ingest_paused_date from recordsets where array_to_string(recordids, ' ') like '%{0}%'".format(key)
    # Fetch all the rows as a list of dictionaries
    rs = [dict(row) for row in db.fetchall(rs_sql)]
    current_rs[key] = rs


In [10]:

print(len(current_rs))

current_rs

354


{'0150c79f-35f3-447f-a40f-24ee9f1fb9d3': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ipt.vertnet.org/eml.do?r=dmnh_eggs',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   'file_link': u'https://ipt.vertnet.org/archive.do?r=dmnh_eggs',
   'first_seen': datetime.datetime(2024, 1, 3, 20, 33, 23, 728602),
   'id': 150642L,
   'ingest': False,
   'ingest_is_paused': False,
   'ingest_paused_date': None,
   'last_seen': datetime.datetime(2024, 3, 20, 16, 6, 33, 801248),
   'name': u'DMNH Eggs - Version 6.1',
   'pub_date': datetime.datetime(2024, 1, 31, 21, 7, 12),
   'publisher_uuid': '842a2bb5-d705-4d6c-8401-abf3ca28c05d',
   'recordids': [u'0150c79f-35f3-447f-a40f-24ee9f1fb9d3'],
   'uuid': None}],
 '01bdf3ea-140a-4d45-a27c-aeca97662aea': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ipt.vertnet.org/eml.do?r=acunhc_ento',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   'file_link': u'htt

In [11]:
#Update the collection

for key in current_rs.keys():
    for item in current_rs[key]:
        for k, v in item.items():
            if k == 'eml_link':
                if v.startswith('http'):
                    print('updating eml_link...')
                    item[k] = v.replace('http://ipt.vertnet.org:8080/ipt', 'https://ipt.vertnet.org')
            if k == 'file_link':
                if v.startswith('http'):
                    print('updating file_link...')
                    item[k] = v.replace('http://ipt.vertnet.org:8080/ipt', 'https://ipt.vertnet.org')
            if k == 'recordids':
                for i, rid in enumerate(v):
                    if rid.startswith('http'):
                        print('updating recordid...')
                        item[k][i] = rid.replace('http://ipt.vertnet.org:8080/ipt', 'https://ipt.vertnet.org')

current_rs

updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating eml_link...
updating file_link...
updating em

{'0150c79f-35f3-447f-a40f-24ee9f1fb9d3': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ipt.vertnet.org/eml.do?r=dmnh_eggs',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   'file_link': u'https://ipt.vertnet.org/archive.do?r=dmnh_eggs',
   'first_seen': datetime.datetime(2024, 1, 3, 20, 33, 23, 728602),
   'id': 150642L,
   'ingest': False,
   'ingest_is_paused': False,
   'ingest_paused_date': None,
   'last_seen': datetime.datetime(2024, 3, 20, 16, 6, 33, 801248),
   'name': u'DMNH Eggs - Version 6.1',
   'pub_date': datetime.datetime(2024, 1, 31, 21, 7, 12),
   'publisher_uuid': '842a2bb5-d705-4d6c-8401-abf3ca28c05d',
   'recordids': [u'0150c79f-35f3-447f-a40f-24ee9f1fb9d3'],
   'uuid': None}],
 '01bdf3ea-140a-4d45-a27c-aeca97662aea': [{'eml_harvest_date': None,
   'eml_harvest_etag': None,
   'eml_link': u'https://ipt.vertnet.org/eml.do?r=acunhc_ento',
   'file_harvest_date': None,
   'file_harvest_etag': None,
   'file_link': u'htt

In [12]:
#Update Records

eml_link = ''
file_link = ''
update_recordid = False
recordid = ''
rs_sql = ''

db.execute("BEGIN")

for key in current_rs.keys():
    for item in current_rs[key]:
        for k, v in item.items():
            if k == 'eml_link':
                if v.startswith('https') and not v.startswith('https://ipt.tacc.utexas.edu'):
                    print('updating eml_link sql...')
                    eml_link = v
            if k == 'file_link':
                if v.startswith('https') and not v.startswith('https://ipt.tacc.utexas.edu'):
                    print('updating file_link sql...')
                    file_link = v
            if k == 'recordids':
                for i, rid in enumerate(v):
                    if rid.startswith('https') and not rid.startswith('https://ipt.tacc.utexas.edu'):
                        print('updating recordid sql...')
                        update_recordid = True
                        recordid = rid
    
    if update_recordid == True:
        rs_sql = "update recordsets set eml_link = '{0}', file_link = '{1}', recordids = string_to_array('{2}', '', null) where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%{3}%'".format(eml_link,file_link,recordid,key)
    else:
        rs_sql = "update recordsets set eml_link = '{0}', file_link = '{1}' where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%{2}%'".format(eml_link,file_link,key)

    print(rs_sql)
    db.execute(rs_sql)
    update_recordid = False

db.commit()





updating eml_link sql...
updating file_link sql...
update recordsets set eml_link = 'https://ipt.vertnet.org/eml.do?r=cuml_sound_film', file_link = 'https://ipt.vertnet.org/archive.do?r=cuml_sound_film' where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%7f6dd0f7-9ed4-49c0-bb71-b2a9c7fed9f1%'
updating eml_link sql...
updating file_link sql...
update recordsets set eml_link = 'https://ipt.vertnet.org/eml.do?r=svdm_paleobotany', file_link = 'https://ipt.vertnet.org/archive.do?r=svdm_paleobotany' where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%db89acb0-b372-4f3d-9ba6-3ab8124a3214%'
updating eml_link sql...
updating file_link sql...
update recordsets set eml_link = 'https://ipt.vertnet.org/eml.do?r=ucm_fish', file_link = 'https://ipt.vertnet.org/archive.do?r=ucm_fish' where publisher_uuid = '842a2bb5-d705-4d6c-8401-abf3ca28c05d' and array_to_string(recordids, ' ') like '%8934ad0c-f